In [38]:
# Calcula la correlación entre un índce de precipitación y uno de otra
# variable en múltiples zonas geográficas.

import pandas as pd
import numpy as np

import xarray as xr

from scipy.stats import linregress

# Datos
path_d = "../results/onset/"
path_r = "../results/onset/graficas/"

# Variable, nivel de presión y región.
vars = [ "u", "v", "gp", "sst", "olr", "sp", #"vidmf",
    "vivfu", "vivfv" ]
levels = [ "925", "200" ]
region = [ "mexico" ]
years = range(1981, 2021)
c = "Cuenca"

# Cantidad de días promediados.
d = 20

# Información de precipitación
df = pd.read_csv(path_d + "pre_daily_CHIRPS.csv", index_col = "time" )
df.index = pd.to_datetime( df.index )

In [39]:
# Crea un índice de onset y withdrawal de acuerdo con
# el umbral para n días seguidos de
# cumplimiento con el criterio.
def pre_onset( umbral = [2.25, 2.25], n = [10, 10], m_i = 4 ):

    df_mean_ind = pd.DataFrame( df.copy(), columns = ["Cuenca"] )

    title = "onset_withdrawal_CHIRPS_diferente.png"

    # Onset.
    # Nombre de las columnas dummy.
    proc = [c]
    for i in range(0, n[0]):
        proc.append(c + "_" + str(i)) 
    # Traslapa los últimos 3 días 
    # y da un valor de 1 a cada día 
    # que cumpla con el umbral.
    for i, p in enumerate(proc[1:]):
        df_mean_ind[p] = df_mean_ind[proc[0]].shift(i)
        df_mean_ind[p].clip( upper = umbral[0],
            inplace = True )
        df_mean_ind[p] = df_mean_ind[p].where(
            df_mean_ind[p] == umbral[0], 0)
        df_mean_ind[p] = df_mean_ind[p].where(
            df_mean_ind[p] < umbral[0], 1)
    # El índice es uno si los n
    # cumplen con el umbral.
    ind = "Index_" + c
    df_mean_ind[ind] = 0
    for i in range(n[0]):
        df_mean_ind[ind] += df_mean_ind[proc[i + 1]]
    df_mean_ind[ind] = df_mean_ind[ind].where( df_mean_ind[ind] == n[0], 0 )
    df_mean_ind[ind] = df_mean_ind[ind].where( df_mean_ind[ind] < n[0], 1 )
    df_mean_ind[ind] = df_mean_ind[ind].where(
        df_mean_ind.index.month.isin( range(5, 12) ), 0 )
    df_mean_ind.drop( proc[1:], axis = 1, inplace = True )
    # Crea una tabla con los valores anuales.
    col = []
    col.append(c + " - Onset")
    col.append(c + " - Withdrawal")
    date = pd.DataFrame()
    for j in years:
        # Solo utilizamos valores entre mayo y noviembre.
        df_year = df_mean_ind[ (df_mean_ind.index.year == j)
            & df_mean_ind.index.month.isin(range(m_i, 12)) ]
        row = []
        # Fechas.
        row.append( df_year.iloc[:, 1].idxmax() )
        row.append( None )
        #row.append( df_year.iloc[::-1, 1].idxmax() )
        # Agregamos la fila al DataFrame.
        date = date.append([row])
    # Recorremos al inicio de la racha de días de precipitación.
    date.iloc[:, 0] = date.iloc[:, 0] - np.timedelta64(n[0], 'D')

    # Withdrawal.
    # Nombre de las columnas dummy.
    proc = [c]
    for i in range(0, n[1]):
        proc.append(c + "_" + str(i)) 
    # Traslapa los últimos 3 días 
    # y da un valor de 1 a cada día 
    # que cumpla con el umbral.
    for i, p in enumerate(proc[1:]):
        df_mean_ind[p] = df_mean_ind[proc[0]].shift(i)
        df_mean_ind[p].clip( upper = umbral[1],
            inplace = True )
        df_mean_ind[p] = df_mean_ind[p].where(
            df_mean_ind[p] == umbral[1], 0)
        df_mean_ind[p] = df_mean_ind[p].where(
            df_mean_ind[p] < umbral[1], 1)
    # El índice es uno si los n
    # cumplen con el umbral.
    ind = "Index_" + c
    df_mean_ind[ind] = 0
    for i in range(n[1]):
        df_mean_ind[ind] += df_mean_ind[proc[i + 1]]
    df_mean_ind[ind] = df_mean_ind[ind].where( df_mean_ind[ind] == n[1], 0 )
    df_mean_ind[ind] = df_mean_ind[ind].where( df_mean_ind[ind] < n[1], 1 )
    df_mean_ind[ind] = df_mean_ind[ind].where(
        df_mean_ind.index.month.isin( range(5, 12) ), 0 )
    df_mean_ind.drop( proc[1:], axis = 1, inplace = True )
    # Crea una tabla con los valores anuales.
    col = []
    name = "Cuenca"
    col.append(name + " - Onset")
    col.append(name + " - Withdrawal")
    date_2 = pd.DataFrame()
    for j in years:
        # Solo utilizamos valores entre mayo y noviembre.
        df_year = df_mean_ind[ (df_mean_ind.index.year == j)
            & df_mean_ind.index.month.isin(range(m_i, 12)) ]
        row = []
        # Fechas.
        row.append( None )
        #row.append( df_year.iloc[:, 1].idxmax() )
        row.append( df_year.iloc[::-1, 1].idxmax() )
        # Agregamos la fila al DataFrame.
        date_2 = date_2.append([row])
    # Recorremos al inicio de la racha de días de precipitación.
    date.iloc[:, 1] = date.iloc[:, 1] - np.timedelta64(n[1], 'D')

    # Unimos onset y withdrawal.
    date.iloc[:, 1] = date_2.iloc[:, 1]

    # Damos formato a las columnas y al índice.
    date.columns = col
    date.index = years
    date.index.name = "Año"

    # Longitud de la temporada.
    date[name + " - Length"] = date.iloc[:, 1] - date.iloc[:, 0]

    # Calculamos el shift necesario para alinear las fechas.
    shift = date.iloc[:, 0].apply(lambda x: x.dayofyear)
    shift_2 = date.iloc[:, 0].apply(lambda x: x.dayofyear)

    for i in [0, 1]:
        date.iloc[:, i] = date.iloc[:, i].apply( 
            lambda x: pd.to_datetime(x) )
        date.iloc[:, i] = date.iloc[:, i].apply(
            lambda x: x.dayofyear )

    return date, shift, shift_2

# Cálcula el onset y withdrawal para una variabe.
def var_onset( date_pre, df, onset = True, vars = [-12.5, 7] , m_i = 5 ):

    umbral = vars[0]
    n = int( vars[1] )

    c = "Cuenca"

    # Nombre de las columnas dummy.
    proc = [c]
    for i in range( 0, n ): proc.append(c + "_" + str(i))

    # Traslapa los últimos 3 días 
    # y da un valor de 1 a cada día 
    # que cumpla con el umbral.
    # Umbral positivo.
    if umbral >= 0:
        for i, p in enumerate(proc[1:]):
            df[p] = df[proc[0]].shift(i)
            df[p].clip( upper = umbral, inplace = True )
            df[p] = df[p].where( df[p] == umbral, 0 )
            df[p] = df[p].where( df[p] < umbral, 1 )
    # Umbral negativo
    if umbral < 0:
        for i, p in enumerate(proc[1:]):
            df[p] = df[proc[0]].shift(i)
            df[p].clip( lower = umbral, inplace = True )
            df[p] = df[p].where( df[p] == umbral, 0 )
            df[p] = df[p].where( df[p] > umbral, 1 )

    # El índice es uno si los n
    # cumplen con el umbral.
    ind = "Index_" + c
    df[ind] = 0
    for i in range( n ):
        df[ind] += df[proc[i + 1]]
    df[ind] = df[ind].where( df[ind] == n, 0 )
    df[ind] = df[ind].where( df[ind] < n, 1 )
    df[ind] = df[ind].where( df.index.month.isin( range(m_i, 12) ), 0 )
    df.drop(proc[1:], axis = 1, inplace = True)

    # Crea una tabla con los valores anuales.
    date = date_pre["Cuenca - Onset"].copy()
    years = range(1981, 2021)

    for j in years:
        # Solo utilizamos valores entre mayo y noviembre.
        df_year = df[ (df.index.year == j)
            & df.index.month.isin(range(m_i, 12)) ]

        # Fechas.
        date[j] = df_year.iloc[:, 1].idxmax()

    # Crea una tabla con los valores anuales.
    col = []
    col.append(c + " - Onset")
    col.append(c + " - Withdrawal")

    date = pd.DataFrame()
    years = range(1981, 2021)

    for j in years:
        # Solo utilizamos valores entre mayo y diciembre.
        df_year = df[ (df.index.year == j)
            & df.index.month.isin(range(m_i, 12)) ]

        row = []

        # Fechas.
        row.append( df_year.iloc[:, 1].idxmax() )
        row.append( df_year.iloc[::-1, 1].idxmax() )

        # Agregamos la fila al DataFrame.
        date = date.append([row])

    # Damos formato a las columnas y al índice.
    date.columns = col
    date.index = years
    date.index.name = "Año"
    # Recorremos al inicio de la racha de días de precipitación.
    date = date - np.timedelta64(n, 'D')

    # Correlación entre índice de precipitación y de viento.
    # Se convierten las fechas a días julianos.
    for i in [0, 1]:
        date.iloc[:, i] = date.iloc[:, i].apply( lambda x: pd.to_datetime(x) )
        date.iloc[:, i] = date.iloc[:, i].apply( lambda x: x.dayofyear )
        
    # Evaluamos ya sea onset o withdrawal.
    if onset: i = 0
    else:     i = 1
    # Regresión lineal de las series.
    linreg = linregress(date.iloc[:, i], date_pre.iloc[:, i])
    # Correlaciones de las series.
    corr = linreg.rvalue

    # Liberamos memoria.
    del date

    return corr

# Evalpua el índice para la variable con combinaciones de n y umbral.
def ind_max( df, ds, onset = True, zone = (-80, -70, 12.5, 17.5),
    n = (1, 21), num = 10 ):
    # Precipitación diaria promedio en toda la cuenca.
    data = ( ds.sel( longitude = slice( zone[0], zone[1] ), 
        latitude = slice( zone[3], zone[2] ) )
        .mean(dim = ["latitude", "longitude"]).to_dataframe() )
    data.columns = ["Cuenca"]

    # Rango de umbrales.
    u = [ np.fix( data.min()[0] * 2 ) / 2,
            np.fix( data.max()[0] * 2 ) / 2, num ]

    # No estamos en una región donde la variable no está definida.
    if not np.isnan( data.min()[0] ):

        # pares de n y umbral.
        c = np.stack( np.meshgrid( np.linspace( *u ), np.arange( *n ) ),
            axis = 2 )
        # matriz de resultados
        res = np.ndarray(shape = c.shape[0:2])

        for i in range(c.shape[0]):
            for j in range(c.shape[1]):
                res[i, j] = var_onset( date_pre = df, df = data,
                    onset = onset, vars = c[i, j] )

        # Regresamos la correlación máxima y los pares de n y umbral.
        return ( res.max(), c[ np.argmax(res) // res.shape[1],
            np.argmax(res) % res.shape[1] ] )

    else: return ( 0, [0, 0] )

# Estudiamos el índice para la variable en diferentes regiones.
def eval(df, onset = True, anom = [True, False],
    v = 0, n = 0, step = 20, num = 10):

    # Variable a utilizar.
    if v > 2: lev = ""
    else: lev = "_" + levels[n]

    # Creamos la matriz de regiones.
    lon = np.arange(-120, -50 + step, step)
    lat = np.arange(0 , 40 + step, step)
    lon = np.stack( [ lon[0:-1], lon[1:] ], axis = 1)
    lat = np.stack( [ lat[0:-1], lat[1:] ], axis = 1)
    a = np.repeat(lon, lat.shape[0], axis = 0)
    b = np.tile(lat, [lon.shape[0], 1])
    bounds = np.concatenate([a, b], axis = 1)

    # Utilizamos datos de anomalías o normales.
    A = []
    if anom[0]: A.append(".")
    if anom[1]: A.append("_anom.")
    for a in A:
        x = []
        y = np.ndarray([0])

        # Cargamos el archivo y lo limpiamos.
        fname = ( "onset_" + vars[v] + lev + "_mean_"
            + str(d) + "_dias" + a + "grib" )
        ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
        if v < 3:
            ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])
        elif v < 6:
            ds = ds.drop_vars(["step", "surface", "valid_time"])
        else:
            ds = ds.drop_vars(["step", "entireAtmosphere", "valid_time"])

        # Corremos para todas las regiones.
        for i in bounds:
            z = ind_max( df = df, ds = ds, onset = onset, zone = i, num = num )
            x.append( z )
            y = np.append( y, [z[0]], axis = 0 )

        # Resultados.
        print( ( f"var: {vars[v]}, lev: {levels[n]}, "
            + f"bounds: {bounds[np.argmax(y)]}, "
            + f"corr: {x[np.argmax(y)][0]:.3f}, "
            + f"umbral: {x[np.argmax(y)][1][0]:.3f}, " 
            + f"n. días: {int(x[np.argmax(y)][1][1])}" ) )

# Evalúa todas una variable para una sola región.
def eval_zona(df, onset = True, anom = [True, False],
    zone = (-80, -70, 12.5, 17.5), v = 0, n = 0, step = 20, num = 10):

    # Variable a utilizar.
    if v > 2: lev = ""
    else: lev = "_" + levels[n]

    # Utilizamos datos de anomalías o normales.
    A = []
    if anom[0]: A.append(".")
    if anom[1]: A.append("_anom.")
    for a in A:

        # Cargamos el archivo y lo limpiamos.
        fname = ( "onset_" + vars[v] + lev + "_mean_"
            + str(d) + "_dias" + a + "grib" )
        ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
        if v < 3:
            ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])
        elif v < 6:
            ds = ds.drop_vars(["step", "surface", "valid_time"])
        else:
            ds = ds.drop_vars(["step", "entireAtmosphere", "valid_time"])

        # Evaluamos el índice.
        z = ind_max( df = df, ds = ds, onset = onset, zone = zone, num = num )

        # Resultados.
        print( ( f"var: {vars[v]}, lev: {levels[n]}, "
            + f"bounds: {zone}, "
            + f"corr: {z[0]:.3f}, "
            + f"umbral: {z[1][0]:.3f}, " 
            + f"n. días: {int(z[1][1])}" ) )

# Evalua para todas las variables.
def check( onset = True, zona = False, anom = [True, False],
    U = [ 3 ], zone = (-80, -70, 12.5, 17.5), step = 20, num = 10 ):
    # Iteramos para cada umbral.
    for u in U:
        print( f"\nUmbral: {u} mm/dia" )
        # Calculamos el índice de precipitación.
        date_pre, shift, shift_2 = pre_onset( umbral = [u, u] )
        # Variables en niveles.
        for v in range(0, 3):
            for n in range(0, 2):
                # Evaluamos todas las zonas o una sola.
                if zona:
                    eval_zona( df = date_pre, onset = onset, anom = anom,
                        zone = zone, v = v, n = n, step = step, num = num )
                else:
                    eval(df = date_pre, onset = onset, anom = anom,
                        v = v, n = n, step = step, num = num)
        # Variables en superficie.
        for v in range(3, 8):
            # Evaluamos todas las zonas o una sola.
            if zona:
                eval_zona(df = date_pre, onset = onset, anom = anom,
                    zone = zone, v = v, step = step, num = num)
            else:
                eval(df = date_pre, onset = onset, anom = anom,
                    v = v, n = n, step = step, num = num)

In [40]:
# Onset.
check()


Umbral: 3 mm/dia
var: u, lev: 925, bounds: [-100  -80    0   20], corr: 0.486, umbral: 1.500, n. días: 9
var: u, lev: 200, bounds: [-60 -40   0  20], corr: 0.560, umbral: -4.889, n. días: 9
var: v, lev: 925, bounds: [-100  -80   20   40], corr: 0.420, umbral: -0.333, n. días: 3
var: v, lev: 200, bounds: [-80 -60  20  40], corr: 0.392, umbral: -5.167, n. días: 1
var: gp, lev: 925, bounds: [-100  -80   20   40], corr: 0.376, umbral: 7961.667, n. días: 10
var: gp, lev: 200, bounds: [-60 -40  20  40], corr: 0.464, umbral: 121667.944, n. días: 15
var: sst, lev: 200, bounds: [-100  -80   20   40], corr: 0.411, umbral: 303.000, n. días: 2
var: olr, lev: 200, bounds: [-120 -100   20   40], corr: 0.433, umbral: -279.722, n. días: 4
var: sp, lev: 200, bounds: [-100  -80   20   40], corr: 0.458, umbral: 99730.556, n. días: 15
var: vivfu, lev: 200, bounds: [-120 -100   20   40], corr: 0.515, umbral: -40.167, n. días: 4
var: vivfv, lev: 200, bounds: [-100  -80    0   20], corr: 0.476, umbral: -35.

In [41]:
# Withdrawal.
check(onset = False)


Umbral: 3 mm/dia
var: u, lev: 925, bounds: [-120 -100    0   20], corr: 0.373, umbral: 0.167, n. días: 8
var: u, lev: 200, bounds: [-80 -60   0  20], corr: 0.337, umbral: -1.444, n. días: 18
var: v, lev: 925, bounds: [-60 -40  20  40], corr: 0.350, umbral: 0.833, n. días: 1
var: v, lev: 200, bounds: [-120 -100   20   40], corr: 0.413, umbral: -1.333, n. días: 1
var: gp, lev: 925, bounds: [-60 -40   0  20], corr: 0.399, umbral: 7888.944, n. días: 14
var: gp, lev: 200, bounds: [-120 -100    0   20], corr: 0.416, umbral: 121560.167, n. días: 8
var: sst, lev: 200, bounds: [-120 -100    0   20], corr: 0.450, umbral: 300.222, n. días: 1
var: olr, lev: 200, bounds: [-60 -40   0  20], corr: 0.382, umbral: -245.833, n. días: 7
var: sp, lev: 200, bounds: [-60 -40  20  40], corr: 0.413, umbral: 101843.167, n. días: 18
var: vivfu, lev: 200, bounds: [-60 -40   0  20], corr: 0.484, umbral: -312.333, n. días: 11
var: vivfv, lev: 200, bounds: [-60 -40  20  40], corr: 0.393, umbral: 64.778, n. días: 6

In [42]:
# Viento zonal, mayor resolución, onset.
u = 3
print( f"Umbral: {u} mm/dia" )
date_pre, shift, shift_2 = pre_onset( umbral = [u, u] )
eval(df = date_pre, onset = True, v = 0, n = 0, step = 10, num = 10)

Umbral: 3 mm/dia
var: u, lev: 925, bounds: [-100  -90    0   10], corr: 0.479, umbral: 2.833, n. días: 8


In [43]:
# Viento zonal, mayor resolución, withdrawal.
u = 3
print( f"Umbral: {u} mm/dia" )
date_pre, shift, shift_2 = pre_onset( umbral = [u, u] )
eval(df = date_pre, onset = False, v = 0, n = 0, step = 10, num = 10)

Umbral: 3 mm/dia
var: u, lev: 925, bounds: [-60 -50  20  30], corr: 0.530, umbral: 0.222, n. días: 6


In [44]:
# Onset, viento zonal, varios umbrales.
for u in [ 2.5, 3, 3.5 ]:
    print( f"Umbral: {u} mm/dia" )
    date_pre, shift, shift_2 = pre_onset( umbral = [u, u] )
    eval(df = date_pre, onset = True, v = 0, n = 0, step = 20, num = 5)

Umbral: 2.5 mm/dia
var: u, lev: 925, bounds: [-100  -80    0   20], corr: 0.370, umbral: 1.250, n. días: 14
Umbral: 3 mm/dia
var: u, lev: 925, bounds: [-100  -80    0   20], corr: 0.458, umbral: 1.250, n. días: 16
Umbral: 3.5 mm/dia
var: u, lev: 925, bounds: [-60 -40   0  20], corr: 0.529, umbral: -7.250, n. días: 10


In [45]:
# Withdrawal, viento zonal, varios umbrales.
for u in [ 2.5, 3, 3.5 ]:
    print( f"Umbral: {u} mm/dia" )
    date_pre, shift, shift_2 = pre_onset( umbral = [u, u] )
    eval(df = date_pre, onset = False, v = 0, n = 0, step = 20, num = 5)

Umbral: 2.5 mm/dia
var: u, lev: 925, bounds: [-60 -40  20  40], corr: 0.468, umbral: 2.750, n. días: 12
Umbral: 3 mm/dia
var: u, lev: 925, bounds: [-60 -40  20  40], corr: 0.441, umbral: 2.750, n. días: 12
Umbral: 3.5 mm/dia
var: u, lev: 925, bounds: [-60 -40  20  40], corr: 0.499, umbral: 2.750, n. días: 14


In [46]:
# Onset, CLLJ.
check( zona = True )


Umbral: 3 mm/dia
var: u, lev: 925, bounds: (-80, -70, 12.5, 17.5), corr: 0.312, umbral: -12.333, n. días: 1
var: u, lev: 200, bounds: (-80, -70, 12.5, 17.5), corr: 0.260, umbral: 13.444, n. días: 9
var: v, lev: 925, bounds: (-80, -70, 12.5, 17.5), corr: 0.326, umbral: 1.611, n. días: 14
var: v, lev: 200, bounds: (-80, -70, 12.5, 17.5), corr: 0.251, umbral: 7.833, n. días: 2
var: gp, lev: 925, bounds: (-80, -70, 12.5, 17.5), corr: 0.103, umbral: 7905.611, n. días: 5
var: gp, lev: 200, bounds: (-80, -70, 12.5, 17.5), corr: 0.236, umbral: 122363.111, n. días: 14
var: sst, lev: 925, bounds: (-80, -70, 12.5, 17.5), corr: 0.397, umbral: 302.500, n. días: 19
var: olr, lev: 925, bounds: (-80, -70, 12.5, 17.5), corr: 0.148, umbral: -282.611, n. días: 14
var: sp, lev: 925, bounds: (-80, -70, 12.5, 17.5), corr: 0.083, umbral: 101213.167, n. días: 7
var: vivfu, lev: 925, bounds: (-80, -70, 12.5, 17.5), corr: 0.309, umbral: -461.722, n. días: 3
var: vivfv, lev: 925, bounds: (-80, -70, 12.5, 17.5),

In [47]:
# Withdrawal, CLLJ.
check( onset = False, zona = True)


Umbral: 3 mm/dia
var: u, lev: 925, bounds: (-80, -70, 12.5, 17.5), corr: 0.304, umbral: -10.278, n. días: 9
var: u, lev: 200, bounds: (-80, -70, 12.5, 17.5), corr: 0.342, umbral: 4.667, n. días: 1
var: v, lev: 925, bounds: (-80, -70, 12.5, 17.5), corr: 0.123, umbral: -0.278, n. días: 14
var: v, lev: 200, bounds: (-80, -70, 12.5, 17.5), corr: 0.185, umbral: 4.278, n. días: 15
var: gp, lev: 925, bounds: (-80, -70, 12.5, 17.5), corr: 0.214, umbral: 7731.778, n. días: 14
var: gp, lev: 200, bounds: (-80, -70, 12.5, 17.5), corr: 0.274, umbral: 121555.556, n. días: 1
var: sst, lev: 925, bounds: (-80, -70, 12.5, 17.5), corr: 0.318, umbral: 301.500, n. días: 20
var: olr, lev: 925, bounds: (-80, -70, 12.5, 17.5), corr: 0.328, umbral: -271.167, n. días: 12
var: sp, lev: 925, bounds: (-80, -70, 12.5, 17.5), corr: 0.215, umbral: 101213.167, n. días: 11
var: vivfu, lev: 925, bounds: (-80, -70, 12.5, 17.5), corr: 0.312, umbral: -316.944, n. días: 20
var: vivfv, lev: 925, bounds: (-80, -70, 12.5, 17.